In [ ]:
# Preflight Check
import sys, platform, subprocess, torch
print('='*60)
print('Preflight Check for FLUX.1-schnell GGUF Model')
print('='*60)
print(f'Python version: {sys.version}')
print(f'Python architecture: {platform.architecture()}')
print(f'OS: {platform.system()} {platform.release()}')
subprocess.run([sys.executable, '-m', 'pip', '--version'])
cuda_available = torch.cuda.is_available()
if cuda_available:
    device_name = torch.cuda.get_device_name(0)
    device_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f'PyTorch sees GPU: {device_name} ({device_mem:.1f} GB)')
else:
    print('No GPU detected, using CPU')


In [ ]:
!git clone --recurse-submodules https://github.com/leejet/stable-diffusion.cpp.git
%cd stable-diffusion.cpp
!mkdir -p build
%cd build
!cmake -DGGML_CUDA=ON ..
!cmake --build . --config Release
%cd ../..
!ls stable-diffusion.cpp/build


In [ ]:
!mkdir -p stable-diffusion.cpp/models
!wget -O stable-diffusion.cpp/models/flux1-schnell-q4_0.gguf https://huggingface.co/leejet/FLUX.1-schnell-gguf/resolve/main/flux1-schnell-q4_0.gguf


In [ ]:
# Search for the sd executable
import os
sd_path = None
for root, dirs, files in os.walk('stable-diffusion.cpp'):
    if 'sd' in files:
        sd_path = os.path.join(root, 'sd')
        break
if sd_path is None:
    raise FileNotFoundError("Could not find the 'sd' executable. Make sure the build completed successfully.")
print(f'Using executable at: {sd_path}')


In [ ]:
import subprocess
import os
model_path = "stable-diffusion.cpp/models/flux1-schnell-q4_0.gguf"
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "result.png")
prompt = "photorealistic bombed out burnt soviet apartment block in Kharkiv"
subprocess.run([
    sd_path,
    '--prompt', prompt,
    '--model', model_path,
    '--output', output_path,
    '--steps', '20',
    '--width', '512',
    '--height', '512'
])
